In [6]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate

In [7]:
ratings = pd.read_csv('/Users/mikeredshaw/Downloads/ratings.csv')
movies = pd.read_csv('/Users/mikeredshaw/Downloads/movies.csv')

In [9]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [11]:
data = pd.merge(ratings,movies, on='movieId')

In [12]:
data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,9,16,4.0,842686699,Casino (1995),Crime|Drama
2,12,16,1.5,1144396284,Casino (1995),Crime|Drama
3,24,16,4.0,963468757,Casino (1995),Crime|Drama
4,29,16,3.0,836820223,Casino (1995),Crime|Drama


In [13]:
data.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [14]:
reader = Reader(rating_scale=(0.5,5))

In [15]:
data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)

In [16]:
data

In [18]:
params = {'name' : 'cosine', 'user_based': False} ##User-based or item based filtering? False for item-based

In [19]:
model = KNNBasic(sim_options=params)

In [21]:
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) #Want RMSE MAE to be as close to 0 as possible.

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9787  0.9788  0.9708  0.9741  0.9672  0.9739  0.0045  
MAE (testset)     0.7600  0.7607  0.7543  0.7579  0.7506  0.7567  0.0038  
Fit time          2.15    2.06    1.86    1.88    1.92    1.97    0.11    
Test time         2.50    2.54    2.34    2.42    2.44    2.45    0.07    


{'test_rmse': array([0.9787119 , 0.97883062, 0.97084916, 0.97413145, 0.96715075]),
 'test_mae': array([0.76000339, 0.76069914, 0.75428132, 0.75791942, 0.75061334]),
 'fit_time': (2.1476969718933105,
  2.058053970336914,
  1.8550350666046143,
  1.8760480880737305,
  1.9210801124572754),
 'test_time': (2.4992828369140625,
  2.5432090759277344,
  2.3397040367126465,
  2.42388916015625,
  2.4408628940582275)}

Almost off by a full star in accuracy

In [23]:
from surprise import accuracy
from surprise.model_selection import train_test_split

In [24]:
trainset, testset = train_test_split(data, test_size=0.2)

In [31]:
model2 = KNNBasic(min_k=5, k=10, sim_options={'name': 'msd', 'user_based': True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9486


0.9486361858702752

In [32]:
model2.predict(668, 16)

Prediction(uid=668, iid=16, r_ui=None, est=3.592207474287737, details={'actual_k': 10, 'was_impossible': False})

In [33]:
movies[movies['movieId']==16]

,movieId,title,genres
15,16,Casino (1995),Crime|Drama


In [37]:
final_dataset  = pd.merge(movies, ratings, on ='movieId')

In [42]:
final_dataset['predicted_rating'] = final_dataset.apply(lambda x :model2.predict(x['userId'], x['movieId']).est, axis=1)

In [43]:
final_dataset.head()

,movieId,title,genres,userId,rating,timestamp,predicted_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895,5.000000
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039,3.948003
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933,4.866552
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810,3.809477
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286,3.850000


In [44]:
from surprise import SVD

In [47]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8746  0.8661  0.8786  0.8606  0.8703  0.8701  0.0063  
Fit time          0.48    0.39    0.42    0.42    0.43    0.43    0.03    
Test time         0.04    0.14    0.04    0.04    0.04    0.06    0.04    


{'test_rmse': array([0.8746105 , 0.866147  , 0.87861919, 0.86063701, 0.87033755]),
 'fit_time': (0.4812350273132324,
  0.392409086227417,
  0.4233708381652832,
  0.4237968921661377,
  0.4286012649536133),
 'test_time': (0.03938603401184082,
  0.13621091842651367,
  0.03771495819091797,
  0.036171913146972656,
  0.035407066345214844)}

In [48]:
trainset = data.build_full_trainset()

In [49]:
svd.fit(trainset)

In [50]:
final_dataset['svd_pred'] = final_dataset.apply(lambda x :svd.predict(x['userId'], x['movieId']).est, axis=1)

In [51]:
final_dataset.head()

,movieId,title,genres,userId,rating,timestamp,predicted_rating,svd_pred
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895,5.000000,4.008321
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039,3.948003,3.518099
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933,4.866552,4.371647
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810,3.809477,3.581849
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286,3.850000,3.844046


In [52]:
ratings = pd.read_csv('/Users/mikeredshaw/Downloads/ratings.csv')
movies = pd.read_csv('/Users/mikeredshaw/Downloads/movies.csv')

In [53]:
data = pd.merge(ratings,movies, on='movieId')

In [54]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [55]:
from sklearn.model_selection import train_test_split

In [61]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [56]:
data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,9,16,4.0,842686699,Casino (1995),Crime|Drama
2,12,16,1.5,1144396284,Casino (1995),Crime|Drama
3,24,16,4.0,963468757,Casino (1995),Crime|Drama
4,29,16,3.0,836820223,Casino (1995),Crime|Drama


In [59]:
data.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
title         object
genres        object
dtype: object

In [63]:
data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,9,16,4.0,842686699,Casino (1995),Crime|Drama
2,12,16,1.5,1144396284,Casino (1995),Crime|Drama
3,24,16,4.0,963468757,Casino (1995),Crime|Drama
4,29,16,3.0,836820223,Casino (1995),Crime|Drama


In [67]:
X = data[{'userId', 'movieId'}]  # Independent variables (all columns except 'DEFAULT')
y = data['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/var/folders/4r/t2pxznrn4dz28d6ymprq1p9c0000gn/T/ipykernel_60500/4118095215.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X = data[{'userId', 'movieId'}]  # Independent variables (all columns except 'DEFAULT')


In [68]:
X, y = make_regression(n_features=4, n_informative=2,random_state=0, shuffle=False)

In [69]:
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X, y)

RandomForestRegressor(max_depth=2, random_state=0)

In [71]:
RandomForestRegressor(...)
print(regr.predict(y_test)

SyntaxError: incomplete input (1183242218.py, line 2)